In [9]:
import os
import sys

print(sys.path)


# Diretório atual
current_dir = os.getcwd()
print(f"Diretório atual: {current_dir}")

# Listar arquivos e pastas dentro do diretório atual
print("Arquivos e pastas no diretório atual:")
print(os.listdir(current_dir))

# Caminho absoluto do arquivo .env que está sendo carregado
dotenv_path = os.path.abspath("config/.env")
print(f"Caminho do arquivo .env: {dotenv_path}")

# Verifica se o arquivo existe
if os.path.exists(dotenv_path):
    print("✅ O arquivo .env foi encontrado!")
else:
    print("❌ O arquivo .env NÃO foi encontrado!")


['/home/jovyan/work', '/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8/org.checkerframework_checker-qual-3.31.0.jar', '/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8/org.postgresql_postgresql-42.6.0.jar', '/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8', '/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip', '/usr/local/spark/python', '/home/jovyan/work', '/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload', '', '/opt/conda/lib/python3.11/site-packages']
Diretório atual: /home/jovyan/work
Arquivos e pastas no diretório atual:
['.git', '.gitignore', 'config', 'docker-compose.yml', 'extract', 'poetry.lock', 'pyproject.toml', 'readme.md', 'tests']
Caminho do arquivo .env: /home/jovyan/work/config/.env
✅ O arquivo .env foi encontrado!


In [10]:
import sys
import os
import pyspark

# Imprime todos os diretórios de busca do Python
print("Caminhos no sys.path:")
for path in sys.path:
    print(path)

# Verifica onde os pacotes estão localizados
print("\nCaminho do pacote 'os':")
print(os.__file__)

print("\nCaminho do pacote 'pyspark':")
print(pyspark.__file__)


Caminhos no sys.path:
/home/jovyan/work
/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8/org.checkerframework_checker-qual-3.31.0.jar
/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8/org.postgresql_postgresql-42.6.0.jar
/tmp/spark-e9d05395-3ed8-404e-8b27-6964879ed933/userFiles-eaaa5d44-a51a-4ae2-bb03-675c7f33aae8
/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip
/usr/local/spark/python
/home/jovyan/work
/opt/conda/lib/python311.zip
/opt/conda/lib/python3.11
/opt/conda/lib/python3.11/lib-dynload

/opt/conda/lib/python3.11/site-packages

Caminho do pacote 'os':
/opt/conda/lib/python3.11/os.py

Caminho do pacote 'pyspark':
/usr/local/spark/python/pyspark/__init__.py


In [11]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from dotenv import load_dotenv
load_dotenv(os.path.join('config', '.env'))

True

In [12]:
# Obtém as variáveis de ambiente
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_HOST = "postgres_db"  # Nome do serviço definido no docker-compose.yml
POSTGRES_PORT = os.getenv("POSTGRES_PORT", "5432")  # Porta padrão do PostgreSQL

# Cria a sessão Spark
spark = SparkSession.builder \
    .appName("PySparkPostgres") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# Define o esquema da tabela
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nome", StringType(), False),
    StructField("idade", IntegerType(), False)
])

# Cria um DataFrame com dados de exemplo
dados = [(1, "Alice", 25), (2, "Bob", 30), (3, "Carlos", 22)]
df = spark.createDataFrame(dados, schema=schema)

# Configurações da conexão JDBC
url = f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
properties = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}

# Escreve os dados no PostgreSQL, criando a tabela "exemplo"
df.write \
    .jdbc(url, "exemplo", mode="overwrite", properties=properties)

print("Tabela 'exemplo' criada com sucesso!")

# Finaliza a sessão Spark
spark.stop()

Tabela 'exemplo' criada com sucesso!
